         <!-- __ _ _ _ __| |_ (_)__ _____
            / _` | '_/ _| ' \| |\ V / -_)
            \__,_|_| \__|_||_|_| \_/\___| -->

Import necessary modules

In [28]:
from bs4 import BeautifulSoup
import requests
import os
from selenium import webdriver
import re

String format:
"/web/20160715062045/http://www.breitbart.com/"
append https://web.archive.org/ to get the full URL

### Inputs:
    Website URL as a string and Date Range in a datetime list

Logic:
A. Go to "https://web.archive.org/web/" + URL
B. For each date in the datetime list:

    i. Find the set of captures on the start of the date range
    ii. Append the midpoint URL to the scrap_URL list
C. For each URL in scrap_URL:

    i. Populate headline_list with the URLs to all the articles on the front page
    ii. Set as a key in the breitbart_raw dictionary
D. For each URL in headline_list:

    i. Add as a key to the headline_dict dictionary
    ii. Set the entry to each key as the raw text from each article

E. For each headline_dict dictionary:
    i. Set as an entry to the appropriate URL key in the breitbart_raw dictionary
    
F. For each URL key in breitbart_raw:
      For each headline key in the headline_dict dictionary:
          i. Clean up the text using standard text cleaning methods
          ii. Compute a bag of words or one-of-k vector to be used as an input in the CNN
### Outputs:
    The scrap_URL list

In [8]:
driver = webdriver.PhantomJS()
driver.get('https://web.archive.org/web/*/www.breitbart.com')

In [21]:
test = requests.get('https://web.archive.org/web/*/www.breitbart.com')
soup = BeautifulSoup(test.text, 'html.parser')

In [50]:
#The set of archive captures on a particular day
day = '20160715'
url = "http://www.breitbart.com/"
archive = "https://web.archive.org"
links = re.findall("/web/"+ day + ".*/" + url, test.text)
#Find the midpoint of the captures (Heuristic for considering only the afternoon)
#archive + links[] will yield the full url to the archived website
links[int(len(links)/2)]

u'https://web.archive.org/web/20160715160510/http://www.breitbart.com/'

In [115]:
#Build the headline_list
test_link = archive + links[int(len(links)/2)]
headline_test = requests.get('https://web.archive.org/web/20160715160510/http://www.breitbart.com/')
soup = BeautifulSoup(headline_test.text, 'html.parser')

articles = soup.find_all("div", class_="article-content")
article_list = list()
for article in articles:
    article_url = re.findall("/web/"+ day + ".*/" + url, str(article.find_all(href = True)))[0].split('">')[0]
    article_list.append(article_url)

In [132]:
#Start scrapping stuff, get the raw text together
#article_test = requests.get(archive + article_list[0])
text_dict = dict()
for article in article_list:
    article_html = requests.get(archive + article)
    soup = BeautifulSoup(article_html.text, 'html.parser')
    content = soup.find_all("div", class_="entry-content")
    try:
        text_dict[article] = content[0].find_all("p")
    except:
        pass